In [25]:
import pandas as pd
import numpy as np

df=pd.read_csv('../data/AQI_Pre.csv')
df.head()

,Date,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,...,City_Kochi,City_Kolkata,City_Lucknow,City_Mumbai,City_Patna,City_Shillong,City_Talcher,City_Thiruvananthapuram,City_Visakhapatnam,Month
0,2015-01-29,83.13,NaN,6.93,28.71,33.72,NaN,6.93,49.52,59.76,...,False,False,False,False,False,False,False,False,False,1
1,2015-01-30,79.84,NaN,13.85,28.68,41.08,NaN,13.85,48.49,97.07,...,False,False,False,False,False,False,False,False,False,1
2,2015-01-31,94.52,NaN,24.39,32.66,52.61,NaN,24.39,67.39,111.33,...,False,False,False,False,False,False,False,False,False,1
3,2015-02-01,135.99,NaN,43.48,42.08,84.57,NaN,43.48,75.23,102.70,...,False,False,False,False,False,False,False,False,False,2
4,2015-02-02,178.33,NaN,54.56,35.31,72.80,NaN,54.56,55.04,107.38,...,False,False,False,False,False,False,False,False,False,2


In [26]:
selected_columns = ['PM2.5', 'PM10', 'NO', 'NO2','NH3','CO','SO2','O3','Month','AQI']
new_df = df[selected_columns]


In [27]:
new_df = new_df.dropna()


In [28]:
new_df.shape

(14345, 10)

In [29]:
from sklearn.model_selection import train_test_split

# Assuming 'AQI' is your target variable
X = new_df.drop(['AQI'], axis=1)
y = new_df['AQI']

# Check class distribution
class_distribution = y.value_counts()
print("Class distribution:\n", class_distribution)

# Identify classes with only one instance
single_instance_classes = class_distribution[class_distribution == 1].index

# Remove instances with single-instance classes
filtered_df = new_df[~y.isin(single_instance_classes)]
X_filtered = filtered_df.drop(['AQI'], axis=1)
y_filtered = filtered_df['AQI']

# Perform train-test-validation split with stratification
X_train, X_temp, y_train, y_temp = train_test_split(X_filtered, y_filtered, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Display the shapes of the resulting sets
print("Training set shape:", X_train.shape, y_train.shape)
print("Validation set shape:", X_val.shape, y_val.shape)
print("Test set shape:", X_test.shape, y_test.shape)


Class distribution:
 AQI
102.0    158
100.0    143
66.0     140
74.0     136
106.0    133
        ... 
613.0      1
561.0      1
716.0      1
453.0      1
489.0      1
Name: count, Length: 512, dtype: int64
Training set shape: (10002, 9) (10002,)
Validation set shape: (2143, 9) (2143,)
Test set shape: (2144, 9) (2144,)


In [30]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor

# Create a random forest model with regularized parameters
rf_model = RandomForestRegressor(n_estimators=100, max_depth=10, min_samples_split=5, min_samples_leaf=2, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Predict on the validation set
y_val_pred = rf_model.predict(X_val)

# Evaluate the model
mse_val = mean_squared_error(y_val, y_val_pred)
mae_val = mean_absolute_error(y_val, y_val_pred)
r2_val = r2_score(y_val, y_val_pred)
rmse_val = mean_squared_error(y_val, y_val_pred, squared=False)

print("Mean Squared Error on Validation Set:", mse_val)
print("Mean Absolute Error on Validation Set:", mae_val)
print("R-squared on Validation Set:", r2_val)
print("Root Mean Squared Error on Validation Set:", rmse_val)


Mean Squared Error on Validation Set: 630.0699344010984
Mean Absolute Error on Validation Set: 15.570712964402807
R-squared on Validation Set: 0.9174162373790393
Root Mean Squared Error on Validation Set: 25.101193883978873


In [31]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Create a simple neural network model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer with 1 neuron for regression
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val), verbose=1)

# Predict on the validation set
y_val_pred_nn = model.predict(X_val)

# Evaluate the model
mse_val_nn = mean_squared_error(y_val, y_val_pred_nn)
mae_val_nn = mean_absolute_error(y_val, y_val_pred_nn)
r2_val_nn = r2_score(y_val, y_val_pred_nn)

print("Neural Network Metrics on Validation Set:")
print("Mean Squared Error:", mse_val_nn)
print("Mean Absolute Error:", mae_val_nn)
print("R-squared:", r2_val_nn)


Epoch 1/50
313/313 [==============================] - 2s 3ms/step - loss: 1635.3195 - val_loss: 1026.8671
Epoch 2/50
313/313 [==============================] - 1s 2ms/step - loss: 940.4798 - val_loss: 923.8137
Epoch 3/50
313/313 [==============================] - 1s 2ms/step - loss: 902.4459 - val_loss: 920.1033
Epoch 4/50
313/313 [==============================] - 1s 2ms/step - loss: 866.7161 - val_loss: 898.6763
Epoch 5/50
313/313 [==============================] - 1s 3ms/step - loss: 846.5050 - val_loss: 870.9129
Epoch 6/50
313/313 [==============================] - 1s 2ms/step - loss: 840.0113 - val_loss: 888.3999
Epoch 7/50
313/313 [==============================] - 1s 3ms/step - loss: 818.1271 - val_loss: 943.4516
Epoch 8/50
313/313 [==============================] - 1s 2ms/step - loss: 817.7126 - val_loss: 860.0359
Epoch 9/50
313/313 [==============================] - 1s 3ms/step - loss: 802.8766 - val_loss: 846.9036
Epoch 10/50
313/313 [==============================] - 1s 2ms/

In [5]:
!pip install xgboost


^C


   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/99.8 MB 3.2 MB/s eta 0:00:32
   ---------------------------------------- 0.1/99.8 MB 1.8 MB/s eta 0:00:55
   ---------------------------------------- 0.2/99.8 MB 1.5 MB/s eta 0:01:07
   ---------------------------------------- 0.2/99.8 MB 1.2 MB/s eta 0:01:20
   ---------------------------------------- 0.2/99.8 MB 1.1 MB/s eta 0:01:34
   ---------------------------------------- 0.3/99.8 MB 923.9 kB/s eta 0:01:48
   ---------------------------------------- 0.3/99.8 MB 923.9 kB/s eta 0:01:48
   ---------------------------------------- 0.3/99.8 MB 923.9 kB/s eta 0:01:48
   ---------------------------------------- 0.3/99.8 MB 681.0 kB/s eta 0:02:27
   ---------------------------------------- 0.3/99.8 MB 703.7 kB/s eta 0:02:22
   ---------------------------------------- 0.3/99.8 MB 703.7 kB/s eta 0:02:22
   ---------------------------------------- 0.3/99.8 MB 635.0 kB/s eta 

In [32]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer

# Create a KNeighborsRegressor model
knn_model = KNeighborsRegressor()

# Define the parameter grid to search
param_grid = {
    'n_neighbors': [3, 5, 7, 9],  # Experiment with different values
    'weights': ['uniform', 'distance'],  # Consider both uniform and distance-based weights
    'p': [1, 2]  # Experiment with different distance metrics (1 for Manhattan, 2 for Euclidean)
}

# Create a scorer for GridSearchCV
scorer = make_scorer(mean_squared_error, greater_is_better=False)

# Create GridSearchCV instance
grid_search = GridSearchCV(knn_model, param_grid, scoring=scorer, cv=5)

# Perform grid search on the training set
grid_search.fit(X_train, y_train)

# Get the best model from the grid search
best_knn_model = grid_search.best_estimator_

# Predict on the validation set using the best model
y_val_pred_best_knn = best_knn_model.predict(X_val)

# Evaluate the best model on the validation set
mse_val_best_knn = mean_squared_error(y_val, y_val_pred_best_knn)
mae_val_best_knn = mean_absolute_error(y_val, y_val_pred_best_knn)
r2_val_best_knn = r2_score(y_val, y_val_pred_best_knn)

print("Best KNN Model from GridSearchCV:")
print("Best Parameters:", grid_search.best_params_)
print("Mean Squared Error:", mse_val_best_knn)
print("Mean Absolute Error:", mae_val_best_knn)
print("R-squared:", r2_val_best_knn)


Best KNN Model from GridSearchCV:
Best Parameters: {'n_neighbors': 9, 'p': 1, 'weights': 'distance'}
Mean Squared Error: 665.5213122379082
Mean Absolute Error: 16.216232383299165
R-squared: 0.9127695973601914


In [33]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Predict on the test set using the previously trained models

# Random Forest Model
y_test_pred_rf = rf_model.predict(X_test)

# Neural Network Model
y_test_pred_nn = model.predict(X_test)

from sklearn.neighbors import KNeighborsRegressor

# Create a KNeighborsRegressor model
knn_model = KNeighborsRegressor(n_neighbors=9, p=1, weights='distance')

# Train the model on the training set
knn_model.fit(X_train, y_train)

# KNN Model
y_test_pred_knn = knn_model.predict(X_test)

# Evaluate models on the test set
mse_rf = mean_squared_error(y_test, y_test_pred_rf)
mae_rf = mean_absolute_error(y_test, y_test_pred_rf)
r2_rf = r2_score(y_test, y_test_pred_rf)

mse_nn = mean_squared_error(y_test, y_test_pred_nn)
mae_nn = mean_absolute_error(y_test, y_test_pred_nn)
r2_nn = r2_score(y_test, y_test_pred_nn)

mse_knn = mean_squared_error(y_test, y_test_pred_knn)
mae_knn = mean_absolute_error(y_test, y_test_pred_knn)
r2_knn = r2_score(y_test, y_test_pred_knn)

# Display results
print("Random Forest Metrics on Test Set:")
print("Mean Squared Error:", mse_rf)
print("Mean Absolute Error:", mae_rf)
print("R-squared:", r2_rf)
print()

print("Neural Network Metrics on Test Set:")
print("Mean Squared Error:", mse_nn)
print("Mean Absolute Error:", mae_nn)
print("R-squared:", r2_nn)
print()

print("KNN Metrics on Test Set:")
print("Mean Squared Error:", mse_knn)
print("Mean Absolute Error:", mae_knn)
print("R-squared:", r2_knn)


67/67 [==============================] - 0s 2ms/step
Random Forest Metrics on Test Set:
Mean Squared Error: 566.2474896489584
Mean Absolute Error: 14.47618352308725
R-squared: 0.9286170194787543

Neural Network Metrics on Test Set:
Mean Squared Error: 675.2250358226562
Mean Absolute Error: 16.565484695914964
R-squared: 0.9148789593584474

KNN Metrics on Test Set:
Mean Squared Error: 585.5153242154483
Mean Absolute Error: 15.042431651865769
R-squared: 0.9261880542564996


In [34]:
import joblib
joblib.dump(rf_model, 'RM2.joblib', compress=('zlib', 3))


['RM2.joblib']

In [22]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13965 entries, 1334 to 24849
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   PM2.5   13965 non-null  float64
 1   PM10    13965 non-null  float64
 2   NO      13965 non-null  float64
 3   NO2     13965 non-null  float64
 4   NOx     13965 non-null  float64
 5   NH3     13965 non-null  float64
 6   CO      13965 non-null  float64
 7   SO2     13965 non-null  float64
 8   O3      13965 non-null  float64
 9   Month   13965 non-null  int64  
dtypes: float64(9), int64(1)
memory usage: 1.2 MB


In [24]:
new_df['NOx'].mean()


33.19973147153598